# LangChain - demo

Este notebook demonstra como construir um pipeline reutilizável com LangChain que:

1. Resume um texto em inglês
2. Traduz o resumo para o português utilizando a interface `Runnable` do LangChain e a composição de prompts.

Essa demonstração ilustra os seguintes conceitos e classes do LangChain:

**Conceitos fundamentais**:

1. **Prompting**
   – Como estruturar entradas para modelos LLM de forma controlada e reutilizável.

2. **Runnable pipeline**
   – Composição modular de etapas de processamento usando `|`.

3. **Encadeamento de componentes (`pipe`)**
   – Conectar prompt → modelo → parser de forma declarativa.

4. **Execução síncrona com `.invoke()`**
   – Chamada direta de um pipeline com entrada como dicionário.

**Classes do LangChain**:

| Classe                      | Finalidade                                                                     |
| --------------------------- | ------------------------------------------------------------------------------ |
| `PromptTemplate`            | Cria e formata prompts com variáveis dinâmicas.                                |
| `StrOutputParser`           | Limpa e interpreta a saída do modelo como string.                              |
| `Runnable` (interface)      | Representa componentes executáveis (`.invoke()`), como prompts, LLMs, parsers. |
| `ChatOpenAI` / `ChatOllama` | Interfaces para comunicação com modelos de linguagem (OpenAI ou Ollama).       |


In [8]:
%run get_llm.py

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import Runnable

# Prompt de sumarização
summary_prompt = PromptTemplate.from_template(
    "Resuma o seguinte texto em uma frase:\n\n{texto}"
)

# Prompt de tradução
translate_prompt = PromptTemplate.from_template(
    "Traduza para o português o seguinte resumo:\n\n{resumo}"
)

# Instancia o modelo (usando sua função já definida)
llm = get_llm()

# Composição das chains
summary_chain: Runnable = summary_prompt | llm 
translate_chain: Runnable = translate_prompt | llm

# Entrada original
artigo = """
Marie Curie was a physicist and chemist who conducted pioneering research on radioactivity.
She achieved unparalleled recognition, becoming the first woman to win a Nobel Prize
and the only person to win Nobel Prizes in both Physics and Chemistry.
"""

# Execução
resumo = summary_chain.invoke({"texto": artigo})
traducao = translate_chain.invoke({"resumo": resumo})

# Resultado final
print("Resumo:", resumo)
print("Tradução:", traducao)

Resumo: content='Marie Curie was a groundbreaking physicist and chemist who achieved unprecedented recognition, becoming the first woman and only individual to win Nobel Prizes in both Physics and Chemistry.' additional_kwargs={} response_metadata={'model': 'gemma3:latest', 'created_at': '2025-07-01T12:14:49.924704006Z', 'done': True, 'done_reason': 'stop', 'total_duration': 16838705396, 'load_duration': 10590350713, 'prompt_eval_count': 64, 'prompt_eval_duration': 2746761645, 'eval_count': 31, 'eval_duration': 3492347382, 'model_name': 'gemma3:latest'} id='run--ec8f2d1a-aaea-4554-902f-694839498737-0' usage_metadata={'input_tokens': 64, 'output_tokens': 31, 'total_tokens': 95}
Tradução: content='Aqui está a tradução do resumo para o português:\n\n"Marie Curie foi uma física e química inovadora que alcançou reconhecimento sem precedentes, tornando-se a primeira mulher e a única pessoa a ganhar Prêmios Nobel em ambas as áreas da Física e da Química."\n\n**Informações Adicionais (metadata

# Descrição do código

## 1. Definição dos prompts

A classe `PromptTemplate` do LangChain serve para criar prompts dinâmicos e reutilizáveis. Ela permite definir um template com variáveis que serão preenchidas em tempo de execução, facilitando a construção de mensagens para modelos LLM.

> Propósito principal: separar a **estrutura do prompt** dos **dados de entrada**, permitindo reutilização e organização do código.

Exemplo:

```python
    from langchain_core.prompts import PromptTemplate

    template = PromptTemplate.from_template("Traduza para o português: {frase}")
    prompt = template.format(frase="Good morning!")

    print(prompt)
    # Saída: "Traduza para o português: Good morning!"
```

Vantagens:

* Facilita a **engenharia de prompts**
* Reduz repetição de código
* Integra bem com chains (`Runnable`) e LLMs

## 2. Construção das cadeias de execução

As linhas abaixo constroem cadeias de execução (chamadas *chains*) que conectam um prompt formatado a um modelo de linguagem (llm), usando o operador | (pipe).


```python
    summary_chain = summary_prompt | llm
    translate_chain = translate_prompt | llm
```

Significado de cada parte:

* `summary_prompt` e `translate_prompt`: objetos `PromptTemplate` que definem o *formato* do prompt para sumarização ou tradução, respectivamente.
* `llm`: o modelo de linguagem obtido via sua função `get_llm()`.
* `|`: operador de **encadeamento** (pipe), que conecta etapas compatíveis do LangChain.


Resultado:

* `summary_chain` é um *runnable* que:

  1. Recebe dados como `{"texto": ...}`
  2. Formata o prompt com `summary_prompt`
  3. Envia o prompt ao modelo `llm`
  4. Retorna a resposta

* `translate_chain` funciona da mesma forma, mas com outro prompt.

Utilidade das cadeias:

* Permitem compor **pipelines modulares** de forma declarativa
* Melhoram legibilidade e organização do código

## 3. Execução das cadeias

```python
resumo = summary_chain.invoke({"texto": artigo})
traducao = translate_chain.invoke({"resumo": resumo})
```

Essas duas linhas são a execução propriamente dita das chains definidas anteriormente com os prompts e o modelo (`summary_chain` e `translate_chain`).

---

# Descrição do resultado

O resultado acima apresenta a resposta do modelo LLM em LangChain com metadados detalhados, incluindo:

1. Conteúdo principal (tradução gerada e justificativas)
2. Informações sobre desempenho (tokens, tempos)
3. Cronometragem da execução no Jupyter


## 1. Conteúdo principal

O primeiro trecho do resultado mostra que o modelo traduziu o parágrafo para o português e adicionou comentários explicativos sobre suas escolhas de tradução.

```text
content="Here's the Portuguese translation of the paragraph:

“Marie Curie foi uma física e química inovadora, conhecida por seu trabalho pioneiro sobre radioatividade. Ela alcançou reconhecimento sem precedentes, tornando-se a primeira mulher a ganhar um Prêmio Nobel e o único indivíduo a receber prêmios Nobel em Física e Química, graças às suas descobertas dos elementos polônio e rádio.”

---

**Notes on the translation:**

*   I’ve aimed for a natural and accurate translation, maintaining the tone and information of the original English.
*   “Groundbreaking” was translated as “inovadora” (innovative).
*   “Unparalleled recognition” was translated as “reconhecimento sem precedentes” (unprecedented recognition).
*   I’ve kept the names of the elements (polonium and radium) as they are commonly used in Portuguese as well."
```

## 2. Metadados da resposta

```python
response_metadata={
  'model': 'gemma3:latest',
  'created_at': '2025-07-01T10:42:05.597910801Z',
  'done': True,
  'done_reason': 'stop',
  'total_duration': 31575438389,
  'load_duration': 87887005,
  'prompt_eval_count': 321,
  'prompt_eval_duration': 7540430110,
  'eval_count': 187,
  'eval_duration': 23946329334,
  'model_name': 'gemma3:latest'
}
```

| Campo                  | Significado                                                                                                                            |
| ---------------------- | -------------------------------------------------------------------------------------------------------------------------------------- |
| `model` / `model_name` | Nome do modelo utilizado. Neste caso, o modelo local `gemma3:latest` via Ollama.                                                       |
| `created_at`           | Timestamp ISO 8601 da execução da inferência. Útil para logs e auditoria.                                                              |
| `done`                 | Booleano indicando se a inferência terminou com sucesso.                                                                               |
| `done_reason`          | Razão pela qual a geração foi encerrada — normalmente `"stop"` (fim natural), mas pode ser `"length"` (limite de tokens) ou `"error"`. |
| `total_duration`       | Tempo total da chamada, em **nanosegundos**: `31.6 segundos`. Engloba todo o ciclo de inferência.                                      |
| `load_duration`        | Tempo gasto para carregar ou inicializar o modelo no runtime: \~88 ms.                                                                 |
| `prompt_eval_count`    | Tokens no **prompt de entrada** (321 tokens), equivalente a `input_tokens`.                                                            |
| `prompt_eval_duration` | Tempo gasto processando o prompt: \~7.5 s.                                                                                             |
| `eval_count`           | Tokens gerados na **resposta do modelo** (187 tokens), equivalente a `output_tokens`.                                                  |
| `eval_duration`        | Tempo de geração da resposta: \~23.9 s.                                                                                                |

Utilidade prática dos metadados acima:

- Ajudam a diagnosticar gargalos (ex: modelo lento, prompt muito longo). Permitem análise de desempenho por modelo, útil para comparar, por exemplo, gemma3 vs llama3.

- Facilitam log e rastreabilidade de execuções em pipelines LangChain.



## 3. Metadados de uso (do LLM)

```python
usage_metadata={
  'input_tokens': 321,
  'output_tokens': 187,
  'total_tokens': 508
}
```

| Campo           | Significado                                                       |
| --------------- | ----------------------------------------------------------------- |
| `input_tokens`  | Quantidade de tokens no prompt fornecido ao modelo (`321` tokens) |
| `output_tokens` | Tokens gerados como resposta pelo modelo (`187` tokens)           |
| `total_tokens`  | Soma total: entrada + saída = `321 + 187 = 508` tokens            |


Esses números podem ser úteis para monitoramento e controle de uso.

- Em modelos pagos (OpenAI , Gemini, Claude), usados para calcular custos em dólares.

- Em modelos locais (ex: Ollama), servem para medir tempo de inferência, tamanho de contexto e desempenho.


## 4. Identificador do run

```python
id='run--a25e68f1-e8a3-4a51-8e44-05e32ac4b3d0-0'
```

Esse é um ID único do `Runnable` ou da `chain.invoke()` executada, útil para debugging e rastreamento.

# Exemplo melhorado

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import Runnable

# Prompt de sumarização
summary_prompt = PromptTemplate.from_template(
    "Resuma o seguinte texto em uma frase:\n\n{texto}"
)

# Prompt de tradução
translate_prompt = PromptTemplate.from_template(
    "Traduza para o português o seguinte resumo:\n\n{resumo}"
)

# Instancia o modelo (usando sua função já definida)
llm = get_llm()

# Parser para limpar a saída do modelo
parser = StrOutputParser()

# Composição das chains
summary_chain: Runnable = summary_prompt | llm | parser
translate_chain: Runnable = translate_prompt | llm | parser

# Entrada original
artigo = """
Marie Curie was a physicist and chemist who conducted pioneering research on radioactivity.
She achieved unparalleled recognition, becoming the first woman to win a Nobel Prize
and the only person to win Nobel Prizes in both Physics and Chemistry.
"""

# Execução
resumo = summary_chain.invoke({"texto": artigo})
traducao = translate_chain.invoke({"resumo": resumo})

# Resultado final
print("Resumo:", resumo)
print("Tradução:", traducao)